### Show t-value and mean/sd calculations produce similar results

This is for the variance calculation, that comes from https://www.jepusto.com/alternative-formulas-for-the-smd/

The version of $V_{d}$ based on a t-value is:

$ V_{d} = d^{2} (\frac{1}{t^2} + \frac{1}{2\nu}) $

which equals:

$ V_{d} = \frac{d^{2}}{t^2} + \frac{d^{2}}{2\nu} $

The version based on se/sd information is:

$ V_{d} = (\frac{se_{b}}{S})^{2} + \frac{d^2}{2\nu} $

Since $\frac{d^2}{2\nu}$ is the same in both equations we should have:

$(\frac{se_{b}}{S})^{2} = \frac{d^{2}}{t^{2}}$

From _Independent groups with differetn variances_:

$(\frac{se_b}{s_{C}})^{2} = \frac{s^2_{T}}{s^2_{C}}\frac{1}{n_{T}} + \frac{1}{n_{C}}$

If $n_{T} \approx n_{C}$ and $S^{2}_{E} \approx S^{@2}_{C}$

then:

$(\frac{se_{b}}{S})^{2} = \frac{S^{2}}{S^{2}} \frac{1}{n} + \frac{1}{n} = \frac{2}{n}$

so, is $\frac{2}{n}  = \frac{d^{2}}{t^{2}}$?

$ d = \frac{\bar{\mu_{T}} - \bar{\mu_{C}}}{S_{p}} $

$ t = \frac{\bar{\mu_{T}} - \bar{\mu_{C}}}{S_{p}\sqrt{\frac{2}{n}}} $ so 

$\frac{d^{2}}{t^{2}} = \frac{ (\bar{\mu_{T}} - \bar{\mu_{C}})^{2} / s_{p}^{2}} { (\bar{\mu_{T}} - \bar{\mu_{C}})^{2} / (s_{p}^{2}) \times \frac{1}{(\frac{2}{n})}}$

All of the $\mu$ and $s_{p}$ terms cancel, leaving:

$ \frac{1}{\frac{1}{\frac{2}{n}}}  = \frac{1}{\frac{n}{2}} = \frac{2}{n}$

So, yes, the equations are consistent and the values should produce consistent estimates if $n_{C} \approx n_{T}$ and $S_{C} \approx S_{T}$

Turning to an example in code...

In [31]:
import numpy as np

# From Channon et al., 2005
# estimate Vd using sd values and using t values

X_c = 67.6
X_t = 66.5
SD_c = 15.8 # set to 16.7 to check exact equivalence of 15.8 for real value 
SD_t = 16.7
N = 25 # for both
df = N-1 # df for control since control SD is divisor
SD_pooled = np.sqrt((((N-1)*(SD_t**2)) + ((N-1)*(SD_c**2)))   /   (N + N - 2))

#####################################

print("Method 1 - using raw values, not t\n")

print("SD pooled: ",SD_pooled)
SE_b_over_S_squared = (SD_t**2)/((SD_c**2) * N) + (1/N)
print("SE_b_over_S_squared: ",SE_b_over_S_squared)

d = (X_t - X_c)/SD_pooled 
print("d = ",d)
print("df = ",df)
V_d1 = SE_b_over_S_squared + ((d**2)/(2*df))
print("Method 1 V_d = ",V_d1)

######################################

print("\n\nMethod 2 - using t values\n")
# we need a 'd' value -- what is this from the paper? (i.e. which effect size
# do they report?) 
# For testing we use the d from above

t_value = (X_t - X_c)/(SD_pooled * np.sqrt(2/N)) # t value will be a given in papers reporting this
print("t = ",t_value)
t2_div_d2 = ((d**2)/(t_value**2))
V_d2 = t2_div_d2 + ((d**2)/(2*df))
print("Method 2 V_d = ",V_d2)


Method 1 - using raw values, not t

SD pooled:  16.25622957515057
SE_b_over_S_squared:  0.08468674891844255
d =  -0.06766636721724606
df =  24
Method 1 V_d =  0.08478213927786711


Method 2 - using t values

t =  -0.23923673558786893
Method 2 V_d =  0.08009539035942455


In [43]:
# try with g values

def SmallSampleCorrection(InputValue):
    CorrectionMultiplier = 1 - (3/(4*InputValue - 1))
    return(CorrectionMultiplier)

# From Channon et al., 2005
# estimate Vd using sd values and using t values

X_c = 67.6
X_t = 66.5
SD_c = 16.7 
SD_t = 16.7 # set to 16.7 to check exact equivalence of 15.8 for real value 
N = 25 # for both
df = N-1 # df for control since control SD is divisor
SD_pooled = np.sqrt((((N-1)*(SD_t**2)) + ((N-1)*(SD_c**2)))   /   (N + N - 2))

#####################################

print("Method 1 - using raw values, not t\n")

print("SD pooled: ",SD_pooled)
SE_b_over_S_squared = (SD_t**2)/((SD_c**2) * N) + (1/N)
print("SE_b_over_S_squared: ",SE_b_over_S_squared)

d = (X_t - X_c)/SD_pooled 
print("d = ",d)
g = SmallSampleCorrection(df)*d
print("g = ",g)
print("df = ",df)
V_d1 = SE_b_over_S_squared + ((g**2)/(2*df))
print("Method 1 V_d = ",V_d1)

######################################

print("\n\nMethod 2 - using t values\n")
# we need a 'd' value -- what is this from the paper? (i.e. which effect size
# do they report?) 
# For testing we use the d from above

SSC = SmallSampleCorrection(df)
print("Small sample correction: ",SSC)
t_value = ((X_t - X_c)/(SD_pooled)) * (1/np.sqrt(2/N))  * SSC # t value will be a given in papers reporting this
print("t = ",t_value)
t2_div_d2 = ((g**2)/(t_value**2)) 
V_d2 = t2_div_d2 + ((g**2)/(2*df))
print("Method 2 V_d = ",V_d2)

Method 1 - using raw values, not t

SD pooled:  16.7
SE_b_over_S_squared:  0.08
d =  -0.06586826347305355
g =  -0.06378821304758871
df =  24
Method 1 V_d =  0.08008476950257927


Method 2 - using t values

Small sample correction:  0.968421052631579
t =  -0.2255253900286109
Method 2 V_d =  0.08008476950257927


In [12]:
def SmallSampleCorrection(InputValue):
    CorrectionMultiplier = 1 - (3/(4*InputValue - 1))
    return(CorrectionMultiplier)

# using Aitkenhead digits forwards
z_diff = -0.33 # mean(PKU_i - control_mean / control_SD) - estimate of g without small sample correction
t_value = -4.015
d=z_diff
g=SmallSampleCorrection(152 + 76 - 2) * d # small sample corrected d = g
N_pku = 153 # 
N_control = 76
df_control = 75
df_pku = 152

Vd = (z_diff**2) * ((1/t_value**2) + (1/(2 * 152)))
Vd             

0.007113712518888703

In [5]:
import numpy as np

SEb = 1 / np.sqrt(N_pku)
SEb

0.08084520834544433

In [6]:
Sc = SEb / np.sqrt(Vd - (d**2 / (2*df_pku)))
Sc

0.9836167015362393

In [7]:
Vd2 = (SEb/Sc)**2 + (d**2 / (2*df_pku))
Vd2

0.007113712518888704

In [13]:
g

-0.3289036544850498